# tests tab-NTV

In [113]:
import pandas as pd
import json
date = '2023-03-03'
date = '2023-04-17'
indic = {'date_irve': date, 'file': 'IRVE_itinerance_residuel' + date + '.csv',
      'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/Analyse/'}
irve = pd.read_csv(indic['chemin'] + indic['file'], sep=',', low_memory=False)
total = len(irve)
check = list(irve.columns)[17:]
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [17, 18, 19, 20, 21]
secondaire = [22, 23, 24, 25, 26, 27, 28]

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-17]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-17]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)

HTTPError: HTTP Error 404: Not Found

In [ ]:
print(indic)
with open('indic.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(indic) + '\n')


In [ ]:
import matplotlib.pyplot as plt

val = { regles[ind-17]: indic[regles[ind-17]] for ind in range(17,29) if indic[regles[ind-17]] > 0 }
val = dict(sorted(val.items(), key=lambda item:item[1], reverse=False))
fig, ax = plt.subplots()
ax.pie(list(val.values()), labels=list(val.keys()))
ax.set_title("Répartition des défaut d'intégrité")
plt.show()  

In [ ]:
nb pdc total
nb pdc incorrects
% incorrects

nb stations
nb stations incorrectes
% incorrects

In [ ]:
popup = [[], []]
locat = [[], []]
for ind, irve in enumerate((irve_p, irve_s)):
    for i in range(len(irve)):
        popup[ind].append( {'id_station': irve['id_station_itinerance'][i], 
                           'id_pdc': irve['id_pdc_itinerance'][i],
                           'contact_operateur': irve['contact_operateur'][i],
                           'nom_enseigne': irve['nom_enseigne'][i],
                           'nom_station': irve['nom_station'][i],
                           'adresse_station': irve['adresse_station'][i],
                           'fichier des écarts': '<a href="https://github.com/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/">IRVE_itinerance_residuel</a>' } )
        coord = json.loads(irve['coordonneesXY'][i])
        coord.reverse()
        locat[ind].append(coord)

In [ ]:
from util_carto import Cart

cart = Cart([50, 2.5])
cart.add_markers(locat[0], popup=popup[0], color='red', group='écarts entités', max_width=250)
cart.add_markers(locat[1], popup=popup[1], color='orange', group='écarts attributs', max_width=250, icon='bug')
cart.show('irve.html')
